In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# my_drive = GoogleDrive(gauth)

In [3]:
# def clear_trash():
#   for a_file in my_drive.ListFile({'q': "trashed = true"}).GetList():
#     # print the name of the file being deleted.
#     try:
#       print(f"the file {a_file['title']}, is about to get deleted permanently.")
#       # delete the file permanently.
#       a_file.Delete()
#     except:
#       pass

In [4]:
# clear_trash()

In [5]:
# !cp /content/drive/MyDrive/TalkLikeMom/conversations-main.zip /content
# !unzip conversations-main.zip

In [2]:
cd /content/conversations-main/Classifier/

/content/conversations-main/Classifier


In [3]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
import sys
sys.path.append('.')

In [6]:
cp ../utils.py .

In [7]:
from utils import load_jsonl, dump_jsonl, set_random_seed

In [8]:
from data_loader import get_task1_conver, get_task2_conver, preprocess

In [9]:
import pandas as pd

In [10]:
df = get_task1_conver("../Task1//annotated_conersations.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=False)
# print(df[0]["text"][0])
pd.concat(df).groupby("label").count()

Loaded 1234 records from ../Task1//annotated_conersations.jsonl
N 974 121 121


,text
label,
1. Close,551
2. Know each other,230
3. Don't know each other,435


In [11]:

df = get_task2_conver("../Task2/annotated/annotated.jsonl", "closeness", skips = [], only_user=False)
# print(df[0]["text"][0])
pd.concat(df).groupby("label").count()

Loaded 2486 records from ../Task2/annotated/annotated.jsonl
N 1700 212 212


,text
label,
1. Close,222
2. Know each other,158
3. Don't know each other,1487
4. Don't like each other,257


In [12]:
df = get_task1_conver("../Task3/annotated/annotated.jsonl", "closeness", skips = [], only_user=False)
# print(df[0]["text"][0])
pd.concat(df).groupby("label").count()

Loaded 1221 records from ../Task3/annotated/annotated.jsonl
N 977 122 122


,text
label,
1. Close,462
2. Know each other,696
3. Don't know each other,52
4. Don't like each other,11


In [17]:
!pip install -q transformers pythainlp datasets evaluate sentencepiece
# !pip install -q accelerate -U
!pip install -q transformers[torch]
# !pip install -q
!pip install -q shap nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.6 MB/s eta 0:00:00


In [13]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import wandb
# from sklearn.metrics import precision_recall_fscore_support, classification_report
# from pythainlp.tokenize import word_tokenize
import torch
import datasets
from datasets import Dataset, DatasetDict
from transformers import DataCollatorWithPadding, DefaultDataCollator
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer



In [14]:
import random
def set_random_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    np.random.RandomState(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) #seed all gpus
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False

In [15]:
from sklearn.utils import compute_class_weight
import torch.nn as nn
import os, shutil
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import mean_squared_error, r2_score
import json

def run_exp(out_dir, df, regressor_configs, seed=42, report="none", model_name = "clicknext/phayathaibert"):

    set_random_seed(seed)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # device = torch.device("cpu")
    print("START")
    print("step 1: load data")
    train, val, test = df

    # train = train.head(100)
    # val = val.head(100)
    # test = test.head(100)

    print("step 2: load tokenizer")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    num_added_toks = tokenizer.add_special_tokens({"additional_special_tokens": ["usr", "sys", "rep"]})

    print("step 3: init data")
    ds = DatasetDict()
    ds['train'] = Dataset.from_pandas(train)
    ds['val'] = Dataset.from_pandas(val)
    ds['test'] = Dataset.from_pandas(test)

    num_labels = 1
    id2label = {1: regressor_configs["label"]}
    label2id = [regressor_configs["label"]]

    def word_tokenize(d, tokenizer=None, label2id=None, max_length=256):
        texts = [preprocess(t) for t in d["text"]]
#         print(texts)
        tokens = tokenizer(texts, padding='max_length', truncation=True, max_length=max_length)
        tokens["label"] = [regressor_configs["label_fn"](label) for label in d["label"]]
        return tokens

    tokenized_ds = ds.map(word_tokenize, batched=True, fn_kwargs={"tokenizer":tokenizer, "label2id": label2id, "max_length":max_length})
    data_collator = DefaultDataCollator()

    print("step 4: load model")
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, id2label=id2label, label2id=label2id);
    model.resize_token_embeddings(len(tokenizer)+1)
    model = model.to(device)

    # print(model)

    if regressor_configs is None:
        metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
        def compute_metrics(eval_pred):
            predictions, labels = eval_pred
            predictions = np.argmax(predictions, axis=1)
            return metrics.compute(predictions=predictions, references=labels, average="macro")
    else:
        label_fn = regressor_configs["label_fn"]
        def compute_metrics(eval_pred):
            predictions, actual = eval_pred
            predictions = predictions.reshape(-1)

            predicted_labels = [label_fn(p) for p in predictions]
            actual_labels = [label_fn(p) for p in actual]
            p, r, f1, _ = precision_recall_fscore_support(actual_labels, predicted_labels, average='macro')

            return {
                "r2_score": r2_score(actual, predictions),
                "mean_squared_error": np.sqrt(mean_squared_error(actual, predictions)),
                "accuracy": accuracy_score(actual_labels, predicted_labels),
                "f1": f1,
                "precision": p,
                "recall": r,
            }


    print("step 5: fine-tune")

    training_args = TrainingArguments(
        output_dir=out_dir,
        warmup_ratio=0.1,
        learning_rate=1e-4,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_strategy="epoch",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        report_to=report,
        metric_for_best_model="f1",
        save_total_limit=2,
        load_best_model_at_end=True,
        push_to_hub=False,
        run_name=out_dir,
    )

    # print(training_args)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_ds["train"],
        eval_dataset=tokenized_ds["val"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    best_ckpt_path = trainer.state.best_model_checkpoint
    print(best_ckpt_path)

    modle_out_path = out_dir+"/best_model"
    if os.path.exists(modle_out_path):
        shutil.rmtree(modle_out_path)

    os.rename(best_ckpt_path, modle_out_path)
    best_ckpt_path = modle_out_path

    for split, d in [("train", train), ("val", val), ("test", test)]:
      o = trainer.predict(tokenized_ds[split])
      predictions = o.predictions.reshape(-1).tolist()
      d["predict"] = predictions


    train.to_csv(modle_out_path+"/train.csv", index=False)
    val.to_csv(modle_out_path+"/val.csv", index=False)
    test.to_csv(modle_out_path+"/test.csv", index=False)

    print("step 6: evaluate")
    eval_train = trainer.evaluate(tokenized_ds["train"])
    eval_test = trainer.evaluate(tokenized_ds["test"])
    eval_val = trainer.evaluate(tokenized_ds["val"])
    print(eval_test)

    with open(f"{modle_out_path}/eval.json", 'w') as fout:
      json.dump({"train": eval_train, "test": eval_test, "val": eval_val}, fout)

    print("DONE")

    return trainer, eval_val, eval_test

## Task1: Train Model

In [16]:
report = "none"
batch_size = 64
max_length = 128
num_epochs = 20

In [21]:
# df[0]["label"].value_counts().loc[['1. Close', '2. Know each other', "3. Don't know each other"]].plot.bar()

In [22]:
# df[1]["label"].value_counts().loc[['1. Close', '2. Know each other', "3. Don't know each other"]].plot.bar()

In [23]:
# df[2]["label"].value_counts().loc[['1. Close', '2. Know each other', "3. Don't know each other"]].plot.bar()

In [17]:
def closeness_label_fn(label):
    if label == '1. Close':
        return 1.0
    elif label =='2. Know each other':
        return 0.0
    elif label == "3. Don't know each other":
        return -1.0
    elif type(label)==str:
        assert(False)

    if label > 0.5:
        return '1. Close'
    elif label < -0.5:
        return "3. Don't know each other"
    else:
        return '2. Know each other'

In [ ]:
best_trainer = None
best_eval_val = None
best_eval_test = None
best_trainer_idx = None

task_name = "py_task1_clse_usr"
for i in range(5):
  df = get_task1_conver("../Task1/annotated_conersations.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
  trainer, eval_val, eval_test = run_exp(f"./Regressors/{task_name}{i}", df, seed=i, report=report, regressor_configs={
      "label": "close",
      "not_label": "not_close",
      "label_fn": closeness_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if abs(eval_val["eval_f1"] - eval_test["eval_f1"]) > 0.15:
    continue

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer
    best_trainer_idx = i

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

Loaded 1234 records from ../Task1/annotated_conersations.jsonl
N 1096 60 60
START
step 1: load data
step 2: load tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


step 3: init data


Map:   0%|          | 0/1096 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

...

step 5: fine-tune


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,0.630700,0.511329,0.393481,0.715073,0.650000,0.537123,0.555393,0.528238
2,0.312800,0.328411,0.610452,0.573072,0.633333,0.603801,0.655387,0.619450
3,0.249700,0.333807,0.604051,0.577761,0.666667,0.629682,0.667293,0.645848
4,0.161800,0.379116,0.550307,0.615724,0.750000,0.655556,0.675468,0.652519
5,0.112200,0.360791,0.572044,0.600658,0.616667,0.590439,0.659921,0.615309
6,0.084900,0.291878,0.653785,0.540258,0.750000,0.692681,0.704229,0.705372
7,0.091100,0.312113,0.629784,0.558671,0.700000,0.666667,0.711994,0.702553
8,0.060200,0.318744,0.621918,0.564574,0.783333,0.685606,0.695833,0.681504
9,0.066400,0.326391,0.612848,0.571306,0.650000,0.618503,0.672249,0.633943
10,0.048200,0.314590,0.626845,0.560883,0.733333,0.664571,0.677434,0.668334


./Regressors/py_task1_clse_usr4/checkpoint-306


step 6: evaluate


{'eval_loss': 0.21142134070396423, 'eval_r2_score': 0.7410286504753077, 'eval_mean_squared_error': 0.45980578660964966, 'eval_accuracy': 0.75, 'eval_f1': 0.6587301587301587, 'eval_precision': 0.6770833333333334, 'eval_recall': 0.6514336917562723, 'eval_runtime': 0.4134, 'eval_samples_per_second': 145.128, 'eval_steps_per_second': 2.419, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.28965017199516296, 'eval_r2_score': 0.6564281357648122, 'eval_mean_squared_error': 0.538191556930542, 'eval_accuracy': 0.75, 'eval_f1': 0.6935566848610327, 'eval_precision': 0.7012438503666574, 'eval_recall': 0.705371520588912, 'eval_runtime': 0.42, 'eval_samples_per_second': 142.848, 'eval_steps_per_second': 2.381, 'epoch': 20.0}
Result[test] {'eval_loss': 0.21142134070396423, 'eval_r2_score': 0.7410286504753077, 'eval_mean_squared_error': 0.45980578660964966, 'eval_accuracy': 0.75, 'eval_f1': 0.6587301587301587, 'eval_precision': 0.6770833333333334, 'eval_recall': 0.6514336917562723, 'e

In [18]:
import os, shutil
def copytree(src, dst, symlinks=False, ignore=None):

  if os.path.isdir(src):
    if not os.path.exists(dst):
        os.makedirs(dst)

    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)
  else:
      dst_dir = "/".join(dst.split("/")[0:-1])
      if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
      shutil.copy2(src, dst)

In [ ]:

for i in range(5):
  src = f"./Regressors/{task_name}{i}/best_model"
  tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}{i}"

  if i==best_trainer_idx:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

    tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}_best"
    copytree(src, tgt, symlinks=False, ignore=None)
  else:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

  print(f"SAVED {i}")


SAVED 0
SAVED 1
SAVED 2
SAVED 3
SAVED 4


In [19]:
import shap
import scipy as sp

import pickle

def load_shap_values(filepath):
    with open(filepath, 'rb') as fin:
        obj = pickle.load(fin)
    return obj

def save_shap_values(filepath, obj):
    with open(filepath, 'wb') as fin:
        pickle.dump(obj, fin)


def get_shapley(df, trainer, shap_path):
    train, val, test = df
    train["split"] = "train"
    val["split"] = "val"
    test["split"] = "test"

    df = pd.concat([train, test, val])
    print("DATA SIZE", len(df))
    def f(x):
        tokenizer = trainer.tokenizer
        model = trainer.model

        # print(type(x.tolist()))
        tv = tokenizer(x.tolist(), padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")
        input_ids = tv["input_ids"].cuda()
        attention_mask = tv["attention_mask"].cuda()
        # tv = tokenizer(x, truncation=True, max_length=max_length).cuda()
        outputs = model(input_ids, attention_mask=attention_mask)[0].detach().cpu().numpy()

        # scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
        # val = sp.special.logit(scores[:,1]) # use one vs rest logit units

        val = outputs.reshape(-1)
        return val

    explainer = shap.Explainer(f, trainer.tokenizer)
    data = {
        "label": df["label"].values,
        "text": [preprocess(t) for t in df["text"].values],
    }

    shap_values = explainer(data)

    save_shap_values(shap_path, {
        "values": shap_values,
        "inputs": data
    })

    return shap_values

In [ ]:
get_shapley(df, best_trainer, f"./ShapleyValues/{task_name}.pkl");

DATA SIZE 1216


PartitionExplainer explainer: 1217it [1:15:08,  3.71s/it]


In [ ]:
copytree(f"./ShapleyValues/{task_name}.pkl", f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValues/{task_name}.pkl", symlinks=False, ignore=None)

In [20]:
def authority1_label_fn(label):
    if label == '0. Very respect':
        return 1.0
    elif label =='1. Respect':
        return 0.0
    elif label == "2. Normal":
        return -1.0

    elif type(label)==str:
        assert(False)

    # [0, 0.33) =>
    # [0.33, 0.66) =>
    # [0.66, 1] =>

    if label > 0.5:
        return '0. Very respect'
    elif label < -0.5:
        return "2. Normal"
    else:
        return '1. Respect'

In [ ]:
best_trainer = None
best_eval_val = None
best_eval_test = None
best_trainer_idx = None

task_name = "py_task1_auth_usr"
for i in range(5):
  df = get_task1_conver("../Task1/annotated_conersations.jsonl", "authority", skips = ["3. Not respect"], only_user=True)
  trainer, eval_val, eval_test = run_exp(f"./Regressors/{task_name}{i}", df, seed=i, report=report, regressor_configs={
      "label": "respect",
      "not_label": "not_respect",
      "label_fn": authority1_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if abs(eval_val["eval_f1"] - eval_test["eval_f1"]) > 0.15:
    continue

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer
    best_trainer_idx = i

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

Loaded 1234 records from ../Task1/annotated_conersations.jsonl
N 1098 61 61
START
step 1: load data
step 2: load tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.4M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,0.906900,0.688997,-0.104117,0.830059,0.245902,0.131579,0.081967,0.333333
2,0.587700,0.763619,-0.223698,0.873853,0.508197,0.286111,0.254589,0.328758
3,0.537400,0.557587,0.106468,0.746717,0.508197,0.348730,0.603175,0.359150
4,0.397400,0.472246,0.243227,0.687202,0.557377,0.483101,0.586743,0.474183
5,0.286300,0.589739,0.054945,0.767945,0.475410,0.470576,0.539216,0.498366
6,0.235400,0.577878,0.073951,0.760183,0.459016,0.467320,0.545479,0.500980
7,0.165900,0.551168,0.116755,0.742407,0.590164,0.513492,0.567568,0.499346
8,0.095700,0.533695,0.144755,0.730544,0.573770,0.524677,0.667965,0.501961
9,0.083800,0.517511,0.170689,0.719383,0.540984,0.478482,0.656046,0.464379
10,0.078600,0.540447,0.133936,0.735151,0.540984,0.469797,0.644788,0.451961


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


./Regressors/py_task1_auth_usr4/checkpoint-306


step 6: evaluate


{'eval_loss': 0.737676203250885, 'eval_r2_score': -0.06473774481683403, 'eval_mean_squared_error': 0.858880877494812, 'eval_accuracy': 0.45901639344262296, 'eval_f1': 0.33709775349119614, 'eval_precision': 0.3159065628476085, 'eval_recall': 0.3988095238095238, 'eval_runtime': 0.4557, 'eval_samples_per_second': 133.848, 'eval_steps_per_second': 2.194, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.5055677890777588, 'eval_r2_score': 0.18982861378108384, 'eval_mean_squared_error': 0.7110329270362854, 'eval_accuracy': 0.6065573770491803, 'eval_f1': 0.5752352787147853, 'eval_precision': 0.691699604743083, 'eval_recall': 0.5519607843137254, 'eval_runtime': 0.4718, 'eval_samples_per_second': 129.3, 'eval_steps_per_second': 2.12, 'epoch': 20.0}
Result[test] {'eval_loss': 0.737676203250885, 'eval_r2_score': -0.06473774481683403, 'eval_mean_squared_error': 0.858880877494812, 'eval_accuracy': 0.45901639344262296, 'eval_f1': 0.33709775349119614, 'eval_precision': 0.3159065628476

In [ ]:
for i in range(5):
  src = f"./Regressors/{task_name}{i}/best_model"
  tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}{i}"

  if i==best_trainer_idx:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

    tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}_best"
    copytree(src, tgt, symlinks=False, ignore=None)
  else:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

  print(f"SAVED {i}")


SAVED 0
SAVED 1
SAVED 2
SAVED 3
SAVED 4


In [ ]:
get_shapley(df, best_trainer, f"./ShapleyValues/{task_name}.pkl");
copytree(f"./ShapleyValues/{task_name}.pkl", f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValues/{task_name}.pkl", symlinks=False, ignore=None)

DATA SIZE 1220


PartitionExplainer explainer: 1221it [1:15:20,  3.71s/it]


In [ ]:
!rm -rf ./Regressors/*

## Task2: Train Model

In [29]:
best_trainer = None
best_eval_val = None
best_eval_test = None
best_trainer_idx = None

_f1 = []
task_name = "py_task2_clas_usr"
for i in range(5):
  df = get_task2_conver("../Task2/annotated/annotated.jsonl", "closeness", seed=i, skips = ["4. Don't like each other"], only_user=True)
  trainer, eval_val, eval_test = run_exp(f"./Regressors/{task_name}{i}", df, seed=i, report=report, regressor_configs={
      "label": "close",
      "not_label": "not_close",
      "label_fn": closeness_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if abs(eval_val["eval_f1"] - eval_test["eval_f1"]) > 0.15:
    continue

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer
    best_trainer_idx = i
    _f1.append(eval_test["eval_f1"])

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test, sum(_f1)/len(_f1))


Loaded 2486 records from ../Task2/annotated/annotated.jsonl
N 1495 186 186
START
step 1: load data
step 2: load tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.4M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,0.709800,0.430103,0.069544,0.655822,0.763441,0.334831,0.321032,0.357407
2,0.356100,0.344895,0.253878,0.587278,0.720430,0.371411,0.349385,0.410648
3,0.236700,0.419805,0.091823,0.647924,0.763441,0.468395,0.617521,0.451810
4,0.115300,0.377217,0.183954,0.614180,0.752688,0.530939,0.621831,0.514394
5,0.077500,0.364464,0.211544,0.603709,0.768817,0.464488,0.569444,0.436742
6,0.057900,0.413875,0.104652,0.643331,0.768817,0.428812,0.542344,0.411069
7,0.043500,0.369422,0.200818,0.607801,0.774194,0.449898,0.605934,0.427736
8,0.039900,0.360151,0.220875,0.600126,0.768817,0.462656,0.573154,0.438258
9,0.032000,0.374335,0.190190,0.611829,0.758065,0.454709,0.538924,0.433628
10,0.026600,0.353068,0.236197,0.594195,0.790323,0.506640,0.655981,0.476221


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


./Regressors/py_task2_clas_usr4/checkpoint-312


step 6: evaluate


{'eval_loss': 0.34167882800102234, 'eval_r2_score': 0.22765632545479986, 'eval_mean_squared_error': 0.5845329761505127, 'eval_accuracy': 0.8118279569892473, 'eval_f1': 0.5079108234448041, 'eval_precision': 0.5813991500490356, 'eval_recall': 0.5024415171473996, 'eval_runtime': 1.3197, 'eval_samples_per_second': 140.939, 'eval_steps_per_second': 2.273, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.3384448289871216, 'eval_r2_score': 0.2678315245748012, 'eval_mean_squared_error': 0.5817601680755615, 'eval_accuracy': 0.7903225806451613, 'eval_f1': 0.5348553502962705, 'eval_precision': 0.6255508673230192, 'eval_recall': 0.5003787878787879, 'eval_runtime': 1.3239, 'eval_samples_per_second': 140.49, 'eval_steps_per_second': 2.266, 'epoch': 20.0}
Result[test] {'eval_loss': 0.34167882800102234, 'eval_r2_score': 0.22765632545479986, 'eval_mean_squared_error': 0.5845329761505127, 'eval_accuracy': 0.8118279569892473, 'eval_f1': 0.5079108234448041, 'eval_precision': 0.58139915004

In [30]:
for i in range(5):
  src = f"./Regressors/{task_name}{i}/best_model"
  tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}{i}"

  if i==best_trainer_idx:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

    tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}_best"
    copytree(src, tgt, symlinks=False, ignore=None)
  else:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

  print(f"SAVED {i}")

SAVED 0
SAVED 1
SAVED 2
SAVED 3
SAVED 4


In [31]:
get_shapley(df, best_trainer, f"./ShapleyValues/{task_name}.pkl");
copytree(f"./ShapleyValues/{task_name}.pkl", f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValues/{task_name}.pkl", symlinks=False, ignore=None)

DATA SIZE 1867


PartitionExplainer explainer: 1868it [1:22:06,  2.64s/it]


In [ ]:
del _f1

In [ ]:
del best_trainer

In [21]:
def authority2_label_fn(label):
    if label == '1. Respect':
        return 1.0
    elif label =='2. Normal':
        return 0.0
    elif label == "3. Not respect":
        return -1.0
    elif type(label)==str:
        assert(False)

    if label > 0.5:
        return '1. Respect'
    elif label < -0.5:
        return "3. Not respect"
    else:
        return '2. Normal'

In [30]:
best_trainer = None
best_eval_val = None
best_eval_test = None
best_trainer_idx = None

task_name = "py_task2_auth_usr"
for i in range(5):
  df = get_task2_conver("../Task2/annotated/annotated.jsonl", "authority", seed=i, skips = [], only_user=True)
  trainer, eval_val, eval_test = run_exp(f"./Regressors/{task_name}{i}", df, seed=i, report=report, regressor_configs={
      "label": "respect",
      "not_label": "not_respect",
      "label_fn": authority2_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if abs(eval_val["eval_f1"] - eval_test["eval_f1"]) > 0.15:
    continue

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer
    best_trainer_idx = i

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)


Loaded 2486 records from ../Task2/annotated/annotated.jsonl
N 1876 234 234
START
step 1: load data
step 2: load tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.4M [00:00<?, ?B/s]

...

step 5: fine-tune


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,0.260700,0.198959,0.405243,0.446048,0.713675,0.653018,0.735863,0.636368
2,0.148000,0.154105,0.539326,0.392562,0.786325,0.736478,0.762525,0.716147
3,0.109200,0.203486,0.391708,0.451094,0.799145,0.752463,0.755843,0.773157
4,0.064100,0.147458,0.559195,0.384003,0.841880,0.793317,0.840148,0.765715
5,0.046600,0.159843,0.522174,0.399803,0.820513,0.789042,0.780280,0.798583
6,0.036700,0.150774,0.549283,0.388296,0.833333,0.786141,0.818459,0.763655
7,0.022100,0.127375,0.619230,0.356897,0.850427,0.808145,0.854217,0.774498
8,0.022200,0.137898,0.587774,0.371346,0.863248,0.811191,0.890264,0.765662
9,0.016700,0.154800,0.537247,0.393447,0.824786,0.785050,0.802998,0.769545
10,0.012600,0.151986,0.545661,0.389853,0.829060,0.790991,0.807161,0.776792


./Regressors/py_task2_auth_usr4/checkpoint-240


step 6: evaluate


{'eval_loss': 0.1366260051727295, 'eval_r2_score': 0.5897624337783491, 'eval_mean_squared_error': 0.36962950229644775, 'eval_accuracy': 0.8247863247863247, 'eval_f1': 0.7476094525353054, 'eval_precision': 0.8031746031746031, 'eval_recall': 0.7141358055992203, 'eval_runtime': 1.69, 'eval_samples_per_second': 138.46, 'eval_steps_per_second': 2.367, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.1378980427980423, 'eval_r2_score': 0.5877739430734459, 'eval_mean_squared_error': 0.37134623527526855, 'eval_accuracy': 0.8632478632478633, 'eval_f1': 0.8111911911911912, 'eval_precision': 0.8902640856018099, 'eval_recall': 0.7656622324224007, 'eval_runtime': 1.7034, 'eval_samples_per_second': 137.373, 'eval_steps_per_second': 2.348, 'epoch': 20.0}
Result[test] {'eval_loss': 0.1366260051727295, 'eval_r2_score': 0.5897624337783491, 'eval_mean_squared_error': 0.36962950229644775, 'eval_accuracy': 0.8247863247863247, 'eval_f1': 0.7476094525353054, 'eval_precision': 0.80317460317460

In [31]:
for i in range(5):
  src = f"./Regressors/{task_name}{i}/best_model"
  tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}{i}"

  if i==best_trainer_idx:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

    tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}_best"
    copytree(src, tgt, symlinks=False, ignore=None)
  else:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

  print(f"SAVED {i}")

SAVED 0
SAVED 1
SAVED 2
SAVED 3
SAVED 4


In [32]:
get_shapley(df, best_trainer, f"./ShapleyValues/{task_name}.pkl");
copytree(f"./ShapleyValues/{task_name}.pkl", f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValues/{task_name}.pkl", symlinks=False, ignore=None)

DATA SIZE 2344


PartitionExplainer explainer: 2345it [1:40:44,  2.58s/it]


In [33]:
!rm -rf ./Regressors/*

In [35]:
# del best_trainer
# del _f1

## Task3: Train Model

In [29]:
best_trainer = None
best_eval_val = None
best_eval_test = None
best_trainer_idx = None

task_name = "py_task3_clse_usr"
for i in range(5):
  df = get_task1_conver("../Task3/annotated/annotated.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
  trainer, eval_val, eval_test = run_exp(f"./Regressors/{task_name}{i}", df, seed=i, report=report, regressor_configs={
      "label": "close",
      "not_label": "not_close",
      "label_fn": closeness_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if abs(eval_val["eval_f1"] - eval_test["eval_f1"]) > 0.15:
    continue

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer
    best_trainer_idx = i

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)


Loaded 1221 records from ../Task3/annotated/annotated.jsonl
N 968 121 121
START
step 1: load data
step 2: load tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


step 3: init data


Map:   0%|          | 0/968 [00:00<?, ? examples/s]

Map:   0%|          | 0/121 [00:00<?, ? examples/s]

...

step 5: fine-tune


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,0.251100,0.289469,-0.119418,0.538023,0.611570,0.492396,0.492165,0.629882
2,0.176600,0.203419,0.213349,0.451020,0.743802,0.593125,0.578531,0.620794
3,0.166800,0.326737,-0.263539,0.571609,0.611570,0.483630,0.503959,0.612572
4,0.138300,0.439706,-0.700405,0.663103,0.570248,0.515149,0.615686,0.558152
5,0.177000,0.198068,0.234044,0.445048,0.735537,0.579605,0.568398,0.594937
6,0.065900,0.203022,0.214885,0.450580,0.768595,0.640557,0.667163,0.629125
7,0.065600,0.190986,0.261429,0.437020,0.752066,0.626631,0.654343,0.612031
8,0.070800,0.238133,0.079106,0.487989,0.710744,0.588329,0.571429,0.663205
9,0.052900,0.202407,0.217265,0.449896,0.768595,0.628934,0.668627,0.603159
10,0.046500,0.198646,0.231807,0.445697,0.768595,0.637290,0.665945,0.620470


./Regressors/py_task3_clse_usr4/checkpoint-240


step 6: evaluate


{'eval_loss': 0.14189620316028595, 'eval_r2_score': 0.5532253762885823, 'eval_mean_squared_error': 0.3766911029815674, 'eval_accuracy': 0.8429752066115702, 'eval_f1': 0.6673421141506247, 'eval_precision': 0.8918271533965699, 'eval_recall': 0.6423280423280423, 'eval_runtime': 0.933, 'eval_samples_per_second': 129.687, 'eval_steps_per_second': 2.144, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.19597932696342468, 'eval_r2_score': 0.242120197561135, 'eval_mean_squared_error': 0.4426954984664917, 'eval_accuracy': 0.7851239669421488, 'eval_f1': 0.6502956738250855, 'eval_precision': 0.6771771771771772, 'eval_recall': 0.6375635616141945, 'eval_runtime': 0.9257, 'eval_samples_per_second': 130.718, 'eval_steps_per_second': 2.161, 'epoch': 20.0}
Result[test] {'eval_loss': 0.14189620316028595, 'eval_r2_score': 0.5532253762885823, 'eval_mean_squared_error': 0.3766911029815674, 'eval_accuracy': 0.8429752066115702, 'eval_f1': 0.6673421141506247, 'eval_precision': 0.8918271533965

In [30]:
for i in range(5):
  src = f"./Regressors/{task_name}{i}/best_model"
  tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}{i}"

  if i==best_trainer_idx:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

    tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}_best"
    copytree(src, tgt, symlinks=False, ignore=None)
  else:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

  print(f"SAVED {i}")

SAVED 0
SAVED 1
SAVED 2
SAVED 3
SAVED 4


In [31]:
get_shapley(df, best_trainer, f"./ShapleyValues/{task_name}.pkl");
copytree(f"./ShapleyValues/{task_name}.pkl", f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValues/{task_name}.pkl", symlinks=False, ignore=None)

DATA SIZE 1210


PartitionExplainer explainer: 1211it [1:15:10,  3.73s/it]


In [36]:
del best_trainer
# del _f1

In [26]:
best_trainer = None
best_eval_val = None
best_eval_test = None
best_trainer_idx = None

task_name = "py_task3_auth_usr"
for i in range(5):
  df = get_task1_conver("../Task3/annotated/annotated.jsonl", "authority", skips = [], only_user=True)
  trainer, eval_test, eval_val = run_exp(f"./Regressors/{task_name}{i}", df, seed=i, report=report, regressor_configs={
      "label": "respect",
      "not_label": "not_respect",
      "label_fn": authority2_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if abs(eval_val["eval_f1"] - eval_test["eval_f1"]) > 0.15:
    continue

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer
    best_trainer_idx = i

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)


Loaded 1221 records from ../Task3/annotated/annotated.jsonl
N 977 122 122
START
step 1: load data
step 2: load tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


step 3: init data


Map:   0%|          | 0/977 [00:00<?, ? examples/s]

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

...

step 5: fine-tune


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,0.217300,0.149983,-0.145968,0.387276,0.885246,0.677987,0.798500,0.708146
2,0.123700,0.110924,0.152467,0.333053,0.909836,0.635218,0.890118,0.596885
3,0.109500,0.059658,0.544176,0.244249,0.934426,0.808968,0.873457,0.794799
4,0.092400,0.061924,0.526861,0.248845,0.942623,0.827243,0.901376,0.797943
5,0.073100,0.072711,0.444438,0.269650,0.942623,0.827243,0.901376,0.797943
6,0.056600,0.077981,0.404172,0.279251,0.926230,0.726916,0.904905,0.681541
7,0.044500,0.087448,0.331839,0.295716,0.934426,0.808968,0.873457,0.794799
8,0.051300,0.056917,0.565118,0.238573,0.959016,0.883842,0.907321,0.872018
9,0.034600,0.043438,0.668105,0.208418,0.959016,0.873658,0.932212,0.827543
10,0.023300,0.053009,0.594974,0.230238,0.959016,0.885733,0.901368,0.872018


./Regressors/py_task3_auth_usr4/checkpoint-160


step 6: evaluate


{'eval_loss': 0.12646521627902985, 'eval_r2_score': 0.07046511195385052, 'eval_mean_squared_error': 0.35561949014663696, 'eval_accuracy': 0.8688524590163934, 'eval_f1': 0.5775623536696272, 'eval_precision': 0.6512371420628301, 'eval_recall': 0.5476190476190476, 'eval_runtime': 0.9003, 'eval_samples_per_second': 135.517, 'eval_steps_per_second': 2.222, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.12646521627902985, 'eval_r2_score': 0.07046511195385052, 'eval_mean_squared_error': 0.35561949014663696, 'eval_accuracy': 0.8688524590163934, 'eval_f1': 0.5775623536696272, 'eval_precision': 0.6512371420628301, 'eval_recall': 0.5476190476190476, 'eval_runtime': 0.9003, 'eval_samples_per_second': 135.517, 'eval_steps_per_second': 2.222, 'epoch': 20.0}
Result[test] {'eval_loss': 0.05300933122634888, 'eval_r2_score': 0.594973963746646, 'eval_mean_squared_error': 0.23023752868175507, 'eval_accuracy': 0.9590163934426229, 'eval_f1': 0.8857326968346024, 'eval_precision': 0.9013684

In [24]:
for i in range(5):
  src = f"./Regressors/{task_name}{i}/best_model"
  tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}{i}"

  if i==best_trainer_idx:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

    tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}_best"
    copytree(src, tgt, symlinks=False, ignore=None)
  else:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

  print(f"SAVED {i}")

<< 0
SAVED 0
SAVED 1
SAVED 2
SAVED 3
SAVED 4


In [25]:

get_shapley(df, best_trainer, f"./ShapleyValues/{task_name}.pkl");
copytree(f"./ShapleyValues/{task_name}.pkl", f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValues/{task_name}.pkl", symlinks=False, ignore=None)

DATA SIZE 1221


PartitionExplainer explainer: 1222it [1:15:54,  3.73s/it]


In [26]:
!rm -rf ./Regressors/*

In [ ]:
del best_trainer
del _f1